In [1]:
import sqlalchemy as db
import pandas as pd
import datetime as date
import os
from dotenv import load_dotenv

Load the **environment**


In [2]:
load_dotenv()
DATABASE = os.getenv('DATABASE')
DATABASE_SERVER = os.getenv('DATABASE_SERVER')

##### Problems loading the driver
url_object = db.engine.URL.create(
    drivername='ODBC Driver 17 for SQL Server',  
    username='user', 
    password='pass',
    host=DATABASE_SERVER,
    database=DATABASE
)
engine = db.create_engine(url_object)
conn = engine.connect()
metadata = db.MetaData()

In [3]:
# Creating the engine connection using another type of string with the driver at the end
# Works without the port, if i dont provide username and password, uses my windows credentials
connection_str = f'mssql://{DATABASE_SERVER}/{DATABASE}?driver=ODBC Driver 17 for SQL Server'
engine = db.create_engine(connection_str)
conn = engine.connect()
metadata = db.MetaData()

In [5]:
# Test a simple select
result = conn.execute(db.text('select * from departments'))

In [6]:
# Check output
result.fetchall()

[(1, 'Engineering', 'Research and Development', datetime.datetime(2008, 5, 25, 0, 0), None),
 (2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 5, 15, 0, 0), None),
 (3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 26, 0, 0), None),
 (4, 'Marketing', 'Sales and Marketing', datetime.datetime(2008, 4, 27, 0, 0), None),
 (5, 'Purchasing', 'Inventory Management', datetime.datetime(2008, 4, 30, 0, 0), None),
 (6, 'Research and Development', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0), None),
 (7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (9, 'Human Resources', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (10, 'Finance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (11, 'Information Services', 'Executive General and Administration', d

In [7]:
# Another way to query
with engine.connect() as connect:
    result = connect.execute(db.text('select * from departments'))
    print(result.all())

[(1, 'Engineering', 'Research and Development', datetime.datetime(2008, 5, 25, 0, 0), None), (2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 5, 15, 0, 0), None), (3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 26, 0, 0), None), (4, 'Marketing', 'Sales and Marketing', datetime.datetime(2008, 4, 27, 0, 0), None), (5, 'Purchasing', 'Inventory Management', datetime.datetime(2008, 4, 30, 0, 0), None), (6, 'Research and Development', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0), None), (7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None), (8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None), (9, 'Human Resources', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None), (10, 'Finance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None), (11, 'Information Services', 'Executive General and Administration', datetime.da

In [8]:
type(result)

sqlalchemy.engine.cursor.LegacyCursorResult

In [9]:
# Using pandas API to query
pd.read_sql(sql='select * from departments', con=conn)

,DepartmentID,Name,GroupName,ModifiedDate,LASTMONTHDATEV3
0,1,Engineering,Research and Development,2008-05-25,None
1,2,Tool Design,Research and Development,2008-05-15,None
2,3,Sales,Sales and Marketing,2008-04-26,None
3,4,Marketing,Sales and Marketing,2008-04-27,None
4,5,Purchasing,Inventory Management,2008-04-30,None
5,6,Research and Development,Research and Development,2008-04-30,None
6,7,Production,Manufacturing,2008-04-30,None
7,8,Production Control,Manufacturing,2008-04-30,None
8,9,Human Resources,Executive General and Administration,2008-04-30,None
9,10,Finance,Executive General and Administration,2008-04-30,None


In [10]:
# Define the metadata of objects, in the future this should be imported
departments = db.schema.Table(
    'departments',
    metadata,
    db.Column('departmentid', db.SMALLINT),
    db.Column('name', db.NVARCHAR(50)), 
    db.Column('groupname', db.NVARCHAR(50)), 
    db.Column('modifieddate', db.DateTime), 
    db.Column('lastmonthdatev3', db.DateTime)
)

In [11]:
# Automatic metadata import
products = db.schema.Table(
    'products', 
    metadata, 
    autoload=True, 
    autoload_with=engine
)

In [12]:
# testing the metadata object
for i in metadata.sorted_tables:
    print(i.name)

departments
products


In [13]:
print(products.columns.keys())

['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock', 'UnitsOnOrder', 'ReorderLevel', 'Discontinued']


In [14]:
print(departments.columns.keys())

['departmentid', 'name', 'groupname', 'modifieddate', 'lastmonthdatev3']


In [15]:
departments.c.groupname

Column('groupname', NVARCHAR(length=50), table=<departments>)

In [16]:
# query = db.select(departments).where(departments = '')
# query = db.sql.expression.Select([departments])#.where(departments.c.groupname=='Manufacturing')
query = departments.select().where(departments.c.groupname=='Manufacturing')
resultcursor = conn.execute(query)
resultcursor.fetchall()

[(7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None)]

In [17]:
# SELECT using pandas API 
# pd.read_sql(sql=query, con=conn)
df_query_test = pd.read_sql(query, conn)
df_query_test

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,7,Production,Manufacturing,2008-04-30,None
1,8,Production Control,Manufacturing,2008-04-30,None


In [18]:
df_query_test.loc[df_query_test['departmentid'] == '8']

,departmentid,name,groupname,modifieddate,lastmonthdatev3


It outputs nothing, let's check the column's type

In [19]:
df_query_test.dtypes

departmentid                int64
name                       object
groupname                  object
modifieddate       datetime64[ns]
lastmonthdatev3            object
dtype: object

In [20]:
departments.columns.lastmonthdatev3

Column('lastmonthdatev3', DateTime(), table=<departments>)

The departmentid column is an integer that's why the .loc method wasn't working

In [21]:
df_query_test.loc[df_query_test['departmentid'] == 7]

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,7,Production,Manufacturing,2008-04-30,None


In [22]:
df_query_test['name'].loc[df_query_test['departmentid'] == 8]

1    Production Control
Name: name, dtype: object

In [23]:
# Change the dataframe 
df_query_test['departmentid'] = df_query_test['departmentid'] + 100

In [24]:
get_date = date.date.today().isoformat()

In [25]:
get_date_month = date.date.today().month

In [26]:
df_query_test[['modifieddate', 'lastmonthdatev3']] = [get_date, get_date_month ]
df_query_test

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,107,Production,Manufacturing,2023-06-16,6
1,108,Production Control,Manufacturing,2023-06-16,6


In [27]:
df_query_test.to_sql('departments', con=conn,if_exists='append', index=False )

-1

It outputted -1 but it worked, let's check the table

In [28]:
pd.read_sql('select * from departments', conn)

,DepartmentID,Name,GroupName,ModifiedDate,LASTMONTHDATEV3
0,1,Engineering,Research and Development,2008-05-25,NaT
1,2,Tool Design,Research and Development,2008-05-15,NaT
2,3,Sales,Sales and Marketing,2008-04-26,NaT
3,4,Marketing,Sales and Marketing,2008-04-27,NaT
4,5,Purchasing,Inventory Management,2008-04-30,NaT
5,6,Research and Development,Research and Development,2008-04-30,NaT
6,7,Production,Manufacturing,2008-04-30,NaT
7,8,Production Control,Manufacturing,2008-04-30,NaT
8,9,Human Resources,Executive General and Administration,2008-04-30,NaT
9,10,Finance,Executive General and Administration,2008-04-30,NaT


The column lastmonthdatev3 was datetime not integer 

In [29]:
departments.update().where(departments.columns.departmentid == 107).values(lastmonthdatev3 = get_date)

It outputted <sqlalchemy.sql.dml.Update object at 0x000002377AD85910>, let's check the table again 

In [30]:
conn.execute(departments.select()).fetchall()

[(1, 'Engineering', 'Research and Development', datetime.datetime(2008, 5, 25, 0, 0), None),
 (2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 5, 15, 0, 0), None),
 (3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 26, 0, 0), None),
 (4, 'Marketing', 'Sales and Marketing', datetime.datetime(2008, 4, 27, 0, 0), None),
 (5, 'Purchasing', 'Inventory Management', datetime.datetime(2008, 4, 30, 0, 0), None),
 (6, 'Research and Development', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0), None),
 (7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (9, 'Human Resources', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (10, 'Finance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (11, 'Information Services', 'Executive General and Administration', d

It didn't work, need to check this more slowly

Let's try an "executemany" update from

In [31]:
update_query = (
    db.update(departments)
    .where(departments.c.departmentid == db.bindparam('id'))
    .values(lastmonthdatev3 = db.bindparam('new_date'))
)
with engine.begin() as connection_multi:
    connection_multi.execute(
        update_query, 
        [{'id': 107, 'new_date': get_date},
        {'id': 108, 'new_date': get_date}])

No output, just a green check, let's check the table again

In [32]:
conn.execute(departments.select()).fetchmany(25)

[(1, 'Engineering', 'Research and Development', datetime.datetime(2008, 5, 25, 0, 0), None),
 (2, 'Tool Design', 'Research and Development', datetime.datetime(2008, 5, 15, 0, 0), None),
 (3, 'Sales', 'Sales and Marketing', datetime.datetime(2008, 4, 26, 0, 0), None),
 (4, 'Marketing', 'Sales and Marketing', datetime.datetime(2008, 4, 27, 0, 0), None),
 (5, 'Purchasing', 'Inventory Management', datetime.datetime(2008, 4, 30, 0, 0), None),
 (6, 'Research and Development', 'Research and Development', datetime.datetime(2008, 4, 30, 0, 0), None),
 (7, 'Production', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (8, 'Production Control', 'Manufacturing', datetime.datetime(2008, 4, 30, 0, 0), None),
 (9, 'Human Resources', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (10, 'Finance', 'Executive General and Administration', datetime.datetime(2008, 4, 30, 0, 0), None),
 (11, 'Information Services', 'Executive General and Administration', d

### IT **WORKED** 

In [33]:
# Now in a prettier way
pd.read_sql(departments.select(), conn)

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,1,Engineering,Research and Development,2008-05-25,NaT
1,2,Tool Design,Research and Development,2008-05-15,NaT
2,3,Sales,Sales and Marketing,2008-04-26,NaT
3,4,Marketing,Sales and Marketing,2008-04-27,NaT
4,5,Purchasing,Inventory Management,2008-04-30,NaT
5,6,Research and Development,Research and Development,2008-04-30,NaT
6,7,Production,Manufacturing,2008-04-30,NaT
7,8,Production Control,Manufacturing,2008-04-30,NaT
8,9,Human Resources,Executive General and Administration,2008-04-30,NaT
9,10,Finance,Executive General and Administration,2008-04-30,NaT


now delete the two new entries 

In [34]:
db.delete(departments).where(departments.c.departmentid.in_([107,108]))

In [35]:
# Now in a prettier way
pd.read_sql(departments.select(), conn)

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,1,Engineering,Research and Development,2008-05-25,NaT
1,2,Tool Design,Research and Development,2008-05-15,NaT
2,3,Sales,Sales and Marketing,2008-04-26,NaT
3,4,Marketing,Sales and Marketing,2008-04-27,NaT
4,5,Purchasing,Inventory Management,2008-04-30,NaT
5,6,Research and Development,Research and Development,2008-04-30,NaT
6,7,Production,Manufacturing,2008-04-30,NaT
7,8,Production Control,Manufacturing,2008-04-30,NaT
8,9,Human Resources,Executive General and Administration,2008-04-30,NaT
9,10,Finance,Executive General and Administration,2008-04-30,NaT


Didnt work, need to get more insights in the where clause

In [36]:
# Making the delete clause
delete_query = db.delete(departments).where(departments.c.departmentid.in_([107,108]))
conn.execute(delete_query)

In [37]:
# Now in a prettier way
pd.read_sql(departments.select(), conn)

,departmentid,name,groupname,modifieddate,lastmonthdatev3
0,1,Engineering,Research and Development,2008-05-25,None
1,2,Tool Design,Research and Development,2008-05-15,None
2,3,Sales,Sales and Marketing,2008-04-26,None
3,4,Marketing,Sales and Marketing,2008-04-27,None
4,5,Purchasing,Inventory Management,2008-04-30,None
5,6,Research and Development,Research and Development,2008-04-30,None
6,7,Production,Manufacturing,2008-04-30,None
7,8,Production Control,Manufacturing,2008-04-30,None
8,9,Human Resources,Executive General and Administration,2008-04-30,None
9,10,Finance,Executive General and Administration,2008-04-30,None


Found out what happened, i was creating the statements but never commited them, need to remember to call the engine.execute() method